# Imports 

In [1]:
## packages
import pandas as pd
import numpy as np
import os
from langdetect import detect, detect_langs

## define constants
GITHUB_DATA_PATH = "../data/raw_data/"

## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Read in addendums data and combine

In [2]:
## read in multi-sheet excel file
all_foia_sheets = pd.read_excel(GITHUB_DATA_PATH + "FOIA_2021-F-05932_raw_data.xlsx",
                               sheet_name= None)

print("There are %s FOIA sheets"% len(all_foia_sheets))

There are 2 FOIA sheets


In [3]:
## stored as dictionary with sheets as different keys; extract to inspect
fy20_addendums = all_foia_sheets['FOIA_2021-F-05932_FY2020']
fy21_q1_addendums = all_foia_sheets['FOIA_2021-F-05932_FY2021_Q1']

In [5]:
## confirm same columns
cols_20_not21 = set(fy20_addendums.columns).difference(fy21_q1_addendums.columns)
cols_21_not20 = set(fy21_q1_addendums.columns).difference(fy20_addendums.columns)

assert len(cols_20_not21) == 0
assert len(cols_21_not20) == 0 

## rowbind original dictionaries, drop fully duplicated rows, and remove index cols
addendum_init = pd.concat(all_foia_sheets).reset_index()
addendum_init2 = addendum_init.drop_duplicates(["CASE_NUMBER", "SECTION_DETAILS"]).copy()
addendum_init2['is_missing_all'] = (addendum_init2.SECTION_NAME.isnull()) & \
                                (addendum_init2.SECTION_NUMBER.isnull()) & \
                                (addendum_init2.SECTION_DETAILS.isnull())

## create filtered addendum df as: 
## (1) drop rows missing all details (later we'll do a left join to all disclosures) and 
## (2) remove levels columns from reset_index()
addendum = addendum_init2.loc[~addendum_init2.is_missing_all,
                             [col for col in addendum_init2.columns
                             if "levels" not in col]]

print("There are %s rows and %s unique job ids"% (addendum.shape[0], 
                                            len(addendum.CASE_NUMBER.unique())))

There are 99615 rows and 13530 unique job ids


# Clean fields

For now, leave unit of analysis as job-section dyad rather than concatenating / filtering to top sections at this phase

## Cleaning section name and section number

In [6]:

## explore relationship between section # and section name
addendum.SECTION_NAME.value_counts().head()
addendum.SECTION_NUMBER.value_counts().head()


Job Duties - null                         3596
Job Requirements                          2357
Job Duties                                1937
Job Duties - undefined                    1734
Inbound/Outbound Transportation - null    1641
Name: SECTION_NAME, dtype: int64

A.8a    35180
B.6     30519
F.2     12266
A.11    10285
F.1      4911
Name: SECTION_NUMBER, dtype: int64

In [7]:
## group by section # and see different section names
addendum.groupby('SECTION_NUMBER').agg({'SECTION_NAME': lambda x: "; ".join(sorted(x.astype(str).unique()))})

## later: use this to clean up section numbers and names --- eg B3 are different variants of drug screening

,SECTION_NAME
SECTION_NUMBER,
1,Contract Impossibility
2,Workers Compensation
3,Job Specifications
4,Transportation and Daily Subsistence
5,Transportation and Daily Subsistence continue..
6,Hours and Earnings Statements
16,JOB DESCRIPTION/REQUIREMENTS AND WORK RULES
-,- Continued from Section C
3.a.,Family Housing


## Detecting Spanish language to figure out which ones to translate

Previous script used a manual list of Spanish keywords

Here, we should:
    
- First detect ones with any Spanish
- Use Google Cloud API to translate those

Started on step 1 with random sample and Eunice can pick up on step 2

In [60]:
sample_add = addendum.sample(n = 200, random_state = 91988)
sample_add.head()


## example true positive in spanish:
## CASE_NUMBER: H-300-20063-372516

## test language detection code on a couple examples
examples = sample_add.loc[sample_add.CASE_NUMBER.isin(["H-300-20063-372516",
                                                     "H-300-19316-139384"])].copy()
examples

### for eunice, not sure if robust enough 
### to deal with multiple languages in same part
### of text so might want to generalize
def detect_onestr(one_str):
    
    ## return list
    res = detect_langs(one_str)
    
    ## transform into a string and split on :
    split_res = str(res[0]).split(":")

    ## return split
    return(split_res)


## add language and probabilities to dataframe
examples['lang'] = [detect_onestr(one_str)[0] for one_str in examples.SECTION_DETAILS]

examples['lang_prob'] = [detect_onestr(one_str)[1] for one_str in examples.SECTION_DETAILS]

examples

## next steps: see github issue

,level_0,level_1,CASE_NUMBER,SECTION_NAME,SECTION_NUMBER,SECTION_DETAILS,is_missing_all
4667,FOIA_2021-F-05932_FY2020,4667,H-300-19316-139384,Job Requirements,B.6,Housing and utilities are provided at no cost ...,False
92429,FOIA_2021-F-05932_FY2021_Q1,6948,H-300-20300-887996,Pay Deductions - Cont. of Pay Deductions,A.11,Employer reserves the right to pay all domesti...,False
53418,FOIA_2021-F-05932_FY2020,53418,H-300-20063-372516,Job Duties - Job Duties Spanish Version,A.8a,"Cosecha de Sandia a mano, el trabajador camina...",False
72347,FOIA_2021-F-05932_FY2020,72347,H-300-20163-646246,Job Duties - Job Description Cont. 2,A.8a,Stake Sorting: Pick up by hand individual stak...,False
79545,FOIA_2021-F-05932_FY2020,79545,H-300-20174-671800,Job Duties - ABANDONO DE EMPLEO O TERMINACIÓN ...,A.8a,Si un trabajador abandona voluntariamente el e...,False


,level_0,level_1,CASE_NUMBER,SECTION_NAME,SECTION_NUMBER,SECTION_DETAILS,is_missing_all
4667,FOIA_2021-F-05932_FY2020,4667,H-300-19316-139384,Job Requirements,B.6,Housing and utilities are provided at no cost ...,False
53418,FOIA_2021-F-05932_FY2020,53418,H-300-20063-372516,Job Duties - Job Duties Spanish Version,A.8a,"Cosecha de Sandia a mano, el trabajador camina...",False


,level_0,level_1,CASE_NUMBER,SECTION_NAME,SECTION_NUMBER,SECTION_DETAILS,is_missing_all,lang,lang_prob
4667,FOIA_2021-F-05932_FY2020,4667,H-300-19316-139384,Job Requirements,B.6,Housing and utilities are provided at no cost ...,False,en,0.9999963623153328
53418,FOIA_2021-F-05932_FY2020,53418,H-300-20063-372516,Job Duties - Job Duties Spanish Version,A.8a,"Cosecha de Sandia a mano, el trabajador camina...",False,es,0.9999960866557156
